<h3>Part One: Create Simpler Dataframe for Holding Articles</h3>

In [10]:
import pandas as pd

In [15]:
articles = pd.read_csv('china_articles_cleaned.csv')
articles.drop(columns=['Unnamed: 0'], inplace=True)

articles.head()

,title,source,content,source_num,regexed,tokens,word_count,title_len,lemmas
0,China urges greater efforts to ease Korean ten...,Global Times (China),China hopes all parties would do more to ease ...,0,china hopes all parties would do more to ease ...,"['china', 'hopes', 'parties', 'would', 'ease',...",276,51,"['china', 'hope', 'party', 'would', 'ease', 't..."
1,Multilateral cooperation involving China best ...,Global Times (China),The US and several Western powers including Au...,0,the us and several western powers including au...,"['us', 'several', 'western', 'powers', 'includ...",273,79,"['u', 'several', 'western', 'power', 'includin..."
2,"Despite pandemic at home, US continues to make...",Global Times (China),Illustration: Liu Rui/GT\n\n\n\nOver the last ...,0,illustration liu rui gt over the last two ...,"['illustration', 'liu', 'rui', 'gt', 'last', '...",377,67,"['illustration', 'liu', 'rui', 'gt', 'last', '..."
3,Cut Chinese firms in eye of the storm a break:...,Global Times (China),"Photo: IC\n\n\n\n\nAccording to Reuters, Chine...",0,photo ic according to reuters chinese le...,"['photo', 'ic', 'according', 'reuters', 'chine...",438,69,"['photo', 'ic', 'according', 'reuters', 'chine..."
4,Finnair launches new flight services for China,Global Times (China),"Lars Olofsson, Greater China sales director at...",0,lars olofsson greater china sales director at...,"['lars', 'olofsson', 'greater', 'china', 'sale...",361,46,"['lars', 'olofsson', 'greater', 'china', 'sale..."


In [16]:
articles.drop(columns=['regexed', 'tokens', 'word_count', 'title_len', 'lemmas'], inplace=True)

articles.to_csv('articles_for_finetuning.csv')

<h3>Part Two: Create .txt Files for Content of Each Source</h3>

In [17]:
outputs_df = pd.read_csv('gpt3_outputs.csv')
articles_df = pd.read_csv('articles_for_finetuning.csv')

In [18]:
# We don't want to fine-tune on the content of articles where we will then use the corresponding headline to generate an output

# To prevent this, first make a list of all the headlines we intend to use later
titles = outputs_df['title'].values

# Then create a to_use field in the df of articles
articles_df['to_use'] = True

# And set it to False if the title matches one of the headlines in our list
for title in titles:
    articles_df['to_use'].loc[articles_df[articles_df['title']==title].index.values] = False

/Users/mrm311/opt/miniconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [19]:
# Let's confirm that the previous stage excluded 125 articles
articles_df['to_use'].describe()

count     15000
unique        2
top        True
freq      14875
Name: to_use, dtype: object

In [21]:
def create_txt_file(source_id):
    lines_list = []
    to_write_df = articles_df[(articles_df['to_use']==True) & (articles_df['source_num']==source_id)]
    for id in to_write_df.index.values:
        lines_list.append(to_write_df['title'].loc[id] + "::" + to_write_df['content'].loc[id])
        
    f = open('source_'+str(source_id)+'.txt','w')
    f.writelines(lines_list)
    f.close()

In [22]:
for i in range(5):
    create_txt_file(i)

<h3> Part Three: Create Fine-Tuned GPT-2 Models

In [ ]:
!git init LanguageModels
!git clone https://www.github.com/huggingface/transformers
!pip install transformers
!pip install datasets

In [ ]:
#I know 5 copy-pastes where the only change is the numbers 0-5 at the end of file name is dumb but
#it may be easier to read for most than bash scripts from a web notebook.
!python transformers/examples/language-modeling/run_clm.py --model_name_or_path gpt2 --train_file 'drive/Shared drives/CyberAI/Ongoing Projects/OpenAI/Proof of Concept Outputs/FineTuning/source_0.txt' --do_train --output_dir 'drive/Shared drives/CyberAI/Ongoing Projects/OpenAI/Proof of Concept Outputs/FineTuning/Tuned-Model_0' --per_device_train_batch_size=1
!python transformers/examples/language-modeling/run_clm.py --model_name_or_path gpt2 --train_file 'drive/Shared drives/CyberAI/Ongoing Projects/OpenAI/Proof of Concept Outputs/FineTuning/source_1.txt' --do_train --output_dir 'drive/Shared drives/CyberAI/Ongoing Projects/OpenAI/Proof of Concept Outputs/FineTuning/Tuned-Model_1' --per_device_train_batch_size=1
!python transformers/examples/language-modeling/run_clm.py --model_name_or_path gpt2 --train_file 'drive/Shared drives/CyberAI/Ongoing Projects/OpenAI/Proof of Concept Outputs/FineTuning/source_2.txt' --do_train --output_dir 'drive/Shared drives/CyberAI/Ongoing Projects/OpenAI/Proof of Concept Outputs/FineTuning/Tuned-Model_2' --per_device_train_batch_size=1
!python transformers/examples/language-modeling/run_clm.py --model_name_or_path gpt2 --train_file 'drive/Shared drives/CyberAI/Ongoing Projects/OpenAI/Proof of Concept Outputs/FineTuning/source_3.txt' --do_train --output_dir 'drive/Shared drives/CyberAI/Ongoing Projects/OpenAI/Proof of Concept Outputs/FineTuning/Tuned-Model_3' --per_device_train_batch_size=1
!python transformers/examples/language-modeling/run_clm.py --model_name_or_path gpt2 --train_file 'drive/Shared drives/CyberAI/Ongoing Projects/OpenAI/Proof of Concept Outputs/FineTuning/source_4.txt' --do_train --output_dir 'drive/Shared drives/CyberAI/Ongoing Projects/OpenAI/Proof of Concept Outputs/FineTuning/Tuned-Model_4' --per_device_train_batch_size=1

<h3> Part Four: Generate News Stories Using Fine-Tuned Models

In [ ]:
model_0 = path+'Tuned-Model_0'
model_1 = path+'Tuned-Model_1'
model_2 = path+'Tuned-Model_2'
model_3 = path+'Tuned-Model_3'
model_4 = path+'Tuned-Model_4'

In [ ]:
untuned_file = path+'untuned_generations.txt'
model_0_file = path+'tuned_0_generations.txt'
model_1_file = path+'tuned_1_generations.txt'
model_2_file = path+'tuned_2_generations.txt'
model_3_file = path+'tuned_3_generations.txt'
model_4_file = path+'tuned_4_generations.txt'

In [ ]:
#For each headline write a story by using each model and add it to a file of stories for that model. 
for id in outputs_df.index.values:
  headline = outputs_df['title'].loc[id]
  headline_for_untuned = headline+'\n'
  headline_for_tuned = headline+'::'
  !python transformers/examples/text-generation/run_generation.py --length 400 --model_type gpt2 --model_name_or_path gpt2 --prompt "$headline_for_untuned" >> "$untuned_file"
  !python transformers/examples/text-generation/run_generation.py --length 400 --model_type gpt2 --model_name_or_path "$model_0" --prompt "$headline_for_untuned" >> "$model_0_file"
  !python transformers/examples/text-generation/run_generation.py --length 400 --model_type gpt2 --model_name_or_path "$model_1" --prompt "$headline_for_tuned" >> "$model_1_file"
  !python transformers/examples/text-generation/run_generation.py --length 400 --model_type gpt2 --model_name_or_path "$model_2" --prompt "$headline_for_tuned" >> "$model_2_file"
  !python transformers/examples/text-generation/run_generation.py --length 400 --model_type gpt2 --model_name_or_path "$model_3" --prompt "$headline_for_tuned" >> "$model_3_file"
  !python transformers/examples/text-generation/run_generation.py --length 400 --model_type gpt2 --model_name_or_path "$model_4" --prompt "$headline_for_tuned" >> "$model_4_file"

<h3> Part Five: Put the news stories in a dataframe for analysis

In [ ]:
def get_articles_from_file(filepath):
  articles_list = []
  f = open(filepath,'r')
  separation_text = f.readline()
  article_text = ''
  for row in f:
    if row == separation_text: #'=== GENERATED SEQUENCE 1 ===\n':
      articles_list.append(article_text)
      article_text = ''
    else:
      article_text += row
  articles_list.append(article_text)
  f.close()
  return articles_list

In [ ]:
outputs_df['untuned_gpt2'] = get_articles_from_file(untuned_file)
outputs_df['ft_source_0'] = get_articles_from_file(model_0_file)
outputs_df['ft_source_1'] = get_articles_from_file(model_1_file)
outputs_df['ft_source_2'] = get_articles_from_file(model_2_file)
outputs_df['ft_source_3'] = get_articles_from_file(model_3_file)
outputs_df['ft_source_4'] = get_articles_from_file(model_4_file)

In [ ]:
outputs_df['untuned_gpt2'] = outputs_df.apply(lambda row: row['untuned_gpt2'].lstrip(row['title']), axis=1)
outputs_df['untuned_gpt2'] = outputs_df.apply(lambda row: row['untuned_gpt2'].lstrip("\n"), axis=1)

outputs_df['ft_source_0'] = outputs_df.apply(lambda row: row['ft_source_0'].lstrip(row['title']), axis=1)
outputs_df['ft_source_0'] = outputs_df.apply(lambda row: row['ft_source_0'].lstrip("\n"), axis=1)
outputs_df['ft_source_0'] = outputs_df.apply(lambda row: row['ft_source_0'].lstrip("::"), axis=1)

outputs_df['ft_source_1'] = outputs_df.apply(lambda row: row['ft_source_1'].lstrip(row['title']), axis=1)
outputs_df['ft_source_1'] = outputs_df.apply(lambda row: row['ft_source_1'].lstrip("\n"), axis=1)
outputs_df['ft_source_1'] = outputs_df.apply(lambda row: row['ft_source_1'].lstrip("::"), axis=1)

outputs_df['ft_source_2'] = outputs_df.apply(lambda row: row['ft_source_2'].lstrip(row['title']), axis=1)
outputs_df['ft_source_2'] = outputs_df.apply(lambda row: row['ft_source_2'].lstrip("\n"), axis=1)
outputs_df['ft_source_2'] = outputs_df.apply(lambda row: row['ft_source_2'].lstrip("::"), axis=1)

outputs_df['ft_source_3'] = outputs_df.apply(lambda row: row['ft_source_3'].lstrip(row['title']), axis=1)
outputs_df['ft_source_3'] = outputs_df.apply(lambda row: row['ft_source_3'].lstrip("\n"), axis=1)
outputs_df['ft_source_3'] = outputs_df.apply(lambda row: row['ft_source_3'].lstrip("::"), axis=1)

outputs_df['ft_source_4'] = outputs_df.apply(lambda row: row['ft_source_4'].lstrip(row['title']), axis=1)
outputs_df['ft_source_4'] = outputs_df.apply(lambda row: row['ft_source_4'].lstrip("\n"), axis=1)
outputs_df['ft_source_4'] = outputs_df.apply(lambda row: row['ft_source_4'].lstrip("::"), axis=1)

In [ ]:
outputs_df.to_csv(path+'outputs_with_gpt2.csv')